# **Código original - Modificado**

# **1. Criar a variavél de ambiente com a chave API Groq**


In [ ]:
from pathlib import Path
from dotenv import load_dotenv
import os
project_root = Path.cwd().parent.parent  # caminho da raíz do projeto

# Carregar o .env
env_path = project_root / '.env'
load_dotenv(env_path)
api_key = os.getenv('GROQ_API_KEY')


# **2. Criar conexão com a API da Groq**

In [ ]:
import os
from groq import Groq


client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[                                                              # Lista de mensagens da conversa
        {
            "role": "user",                                                 #indica que é mensagem do usuário
            "content": "Explain the importance of fast language models",    # o Texto da pergunta
        }
    ],
    model="llama-3.3-70b-versatile",                                        # Qual modelo utilizar
)

print(chat_completion.choices[0].message.content)

# choices[0]: primeira (e única) resposta gerada
# .message.content: extrai só o texto da resposta

# **3. Testar chamada de REST API EXTERNAS**

In [ ]:
# ============================================================================
# VALIDAÇÃO DE FERRAMENTAS EXTERNAS: REST Countries
# ============================================================================

import requests
from typing import Dict

# ============================================================================
# CONFIGURAÇÃO
# ============================================================================

PAIS = "Brazil"

# Coordenadas de bounding box para cidades principais
# Formato: (sul, oeste, norte, leste)

# ============================================================================
# FERRAMENTA 1: REST Countries API
# ============================================================================

def get_country_info(country_name: str) -> Dict: #função que captura os dados 
    """
    Busca informações básicas de um país
    """
    url = f"https://restcountries.com/v3.1/name/{country_name}"
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()[0]

        result = {
            "country": data['name']['common'],
            "capital": data['capital'][0] if data.get('capital') else "N/A",
            "population": data['population'],
            "region": data['region'],
            "subregion": data.get('subregion', 'N/A'),
            "languages": list(data.get('languages', {}).values()),
            "currencies": {
                code: info['name'] + f" ({info.get('symbol', '')})"
                for code, info in data.get('currencies', {}).items()
            },
            "ddi": data['idd']['root'] + (data['idd'].get('suffixes', [''])[0]),
            "area_km2": data.get('area', 0),
            "timezones": data.get('timezones', []),
            "borders": data.get('borders', [])
        }
        
        return result
        
    except Exception as e:
        return {"error": str(e)}


# ============================================================================
# EXECUÇÃO DOS TESTES
# ============================================================================

print("=" * 80)
print("VALIDACAO DE FERRAMENTAS EXTERNAS")
print("=" * 80)
print(f"\nPais: {PAIS}")

# ----------------------------------------------------------------------------
# TESTE 1: REST Countries API
# ----------------------------------------------------------------------------

print("-" * 80)
print("TESTE 1: REST Countries API")
print("-" * 80)

country_data = get_country_info(PAIS)

if "error" in country_data:
    print(f"ERRO: {country_data['error']}")
else:
    print(f"Status: OK")
    print(f"\nPais: {country_data['country']}")
    print(f"Capital: {country_data['capital']}")
    print(f"Populacao: {country_data['population']:,} habitantes")
    print(f"Regiao: {country_data['region']} ({country_data['subregion']})")
    print(f"Idiomas: {', '.join(country_data['languages'])}")
    
    currencies = [f"{code}: {name}" for code, name in country_data['currencies'].items()]
    print(f"Moedas: {', '.join(currencies)}")
    
    print(f"DDI: {country_data['ddi']}")
    print(f"Area: {country_data['area_km2']:,.0f} km2")
    print(f"Fusos: {', '.join(country_data['timezones'])}")
    
    if country_data['borders']:
        print(f"Fronteiras: {', '.join(country_data['borders'])}")



In [ ]:
# ============================================================================
# TESTE SIMPLES: Valor do Real (BRL) em relação ao Dólar (USD)
# ============================================================================

import requests

# API Frankfurter - SEM API KEY!
url = "https://api.frankfurter.dev/v1/latest?base=USD&symbols=BRL"

# Fazer a requisição
response = requests.get(url)
data = response.json()

# Mostrar resultado
print("=" * 50)
print("COTACAO DO REAL")
print("=" * 50)
print(f"Data: {data['date']}")
print(f"1 USD = {data['rates']['BRL']} BRL")
print("=" * 50)

# **4. Criar a Classe do Agente**

In [ ]:
class Agent:
    def __init__(self, client, system): #Construtor, recebe os argumentos que serão os atributos do objeto
        self.client= client             #Conexão com API Groq,
        self.system= system             #Prompt que define o comportamento do agente | Define personalidade/função (ex: "Você é um assistente técnico")
        self.messages= []               #Armazenar o histórico da conversa
        if self.system is not None:
            self.messages.append({"role": "system", "content": self.system})

    def __call__(self, message=''): #Permite usar a classe como função
        if message:             
            self.messages.append({"role": "user", "content": message})  #Adiciona a pergunta (input) do usario,
        result= self.execute()                                          #Chamada da API do Groq
        self.messages.append({"role": "assistant", "content": result})  #Salva a resposta do modelo, resposta com base no histórico
        return result
    
    def execute(self):
        completion= client.chat.completions.create(
        messages=self.messages,                         #Envia todo o histórico
        model="llama-3.3-70b-versatile",                # Qual modelo está utilizando
    )
        return completion.choices[0].message.content

# **5. Implementando Padrão ReAct**
  
    
    1. Thought   → Raciocínio sobre o problema

    2. Action    → Executa ferramenta

    3. PAUSE     → Aguarda resultado

    4. Observation → Recebe resultado da ferramenta
    
    5. (Volta ao Thought ou dá Answer final)
    


**5.1 System Prompt**

In [ ]:
# ============================================================================
# VARIÁVEL DE CONTROLE DE ITERAÇÕES
# ============================================================================

iter_max = int(input("Digite o numero maximo de iteracoes pretendido: "))

# ============================================================================
# SYSTEM PROMPT ADAPTATIVO - VERSÃO FINAL CORRIGIDA
# ============================================================================

system_prompt = f"""You are a Country Comparison Assistant. You help users compare their current country with a destination country.

CRITICAL: You have MAXIMUM {iter_max} iterations to complete the task.

LANGUAGE DETECTION (CRITICAL):
Step 1: Analyze the country names in the user's question:
- "Brasil", "Japão", "França", "Alemanha", "México" → PORTUGUESE
- "Brazil", "Japan", "France", "Germany", "Mexico" → ENGLISH
- "Brésil", "Japon", "France", "Allemagne" → FRENCH
- "Brasil", "Japón", "Francia", "Alemania" → SPANISH

Step 2: If country names don't clearly indicate language, check question words:
- "Estou", "me fale", "sobre", "gostaria" → PORTUGUESE
- "I'm in", "tell me", "about", "would like" → ENGLISH

Step 3: DEFAULT RULE:
- If ANY Portuguese indicator found → respond in PORTUGUESE
- Only use English if clearly English input with English country names

IMPORTANT: 
- ALWAYS call APIs with English country names (translate internally)
- "Brasil" in input → use "Brazil" for API → respond in Portuguese
- "Brazil" in input → use "Brazil" for API → respond in English

Handle typos: "Mexic" → "Mexico", "Japao" → "Japan", "Lituania" → "Lithuania"

ITERATION BUDGET: {iter_max} iterations total
- Plan strategically to fit within budget
- Prioritize: country info > currency > timezone > answer
- Always deliver an answer, even if partial

PLANNING STRATEGY:

IF {iter_max} >= 5:
  Standard: country1 → country2 → currency → timezone → Answer
  
IF {iter_max} == 3-4:
  Reduced: country1 → country2 → currency → Answer (calculate timezone manually)
  
IF {iter_max} == 2:
  Minimal: country1 → country2 → Answer (calculate currency and timezone manually using data from countries)
  
IF {iter_max} == 1:
  Emergency: Output Answer with general knowledge only

WORKFLOW:
You run in Thought, Action, PAUSE, Observation loop.

Available actions:

get_country_info:
e.g. get_country_info: Brazil
Returns: capital, population, language, currency code, timezone, area

convert_currency:
e.g. convert_currency: BRL, JPY
Returns: exchange rate (use only if iterations allow)

calculate_timezone_diff:
e.g. calculate_timezone_diff: UTC-03:00, UTC+09:00
Returns: time difference (use only if iterations allow)

MANUAL CALCULATION FALLBACKS:
If you run out of iterations or tools fail, YOU CAN calculate manually:

Timezone calculation (simple math):
- UTC-03:00 to UTC+09:00 = 12 hours difference
- Formula: destination_offset - origin_offset = difference
- Example: (+9) - (-3) = +12 hours
- If 12:00 in origin, add difference to get destination time

Currency: If API fails, state "Conversão indisponível"

CRITICAL RULES:
1. Never repeat same tool call
2. Count iterations mentally  
3. On iteration {iter_max}, MUST output Answer
4. Use fallback calculations when needed
5. Track detected language from first analysis

MANDATORY ANSWER FORMAT:

Portuguese format:
Answer: Comparação [País1] vs [País2]

Moeda: 1 [Moeda1] = X,XX [Moeda2]
(ou "Conversão de moeda indisponível")

Horário: Quando são 12h em [Capital1], são XXh em [Capital2] (X horas à frente/atrás)
(ALWAYS include this - calculate manually if needed)

Comparação:
- [País2] é Xx maior/menor em área (XXK km² vs XXK km²)
- [País1/2] tem Xx mais habitantes (XXM vs XXM)
- Capitais: [Capital1] vs [Capital2]
- Idiomas: [Idioma1] vs [Idioma2]

English format:
Answer: [Country1] vs [Country2] Comparison

Currency: 1 [Currency1] = X.XX [Currency2]
(or "Currency conversion unavailable")

Time: When it's noon in [Capital1], it's XXh in [Capital2] (X hours ahead/behind)
(ALWAYS include this - calculate manually if needed)

Comparison:
- [Country2] is Xx larger/smaller (XXK km² vs XXK km²)
- [Country1/2] has Xx more people (XXM vs XXM)
- Capitals: [Capital1] vs [Capital2]
- Languages: [Language1] vs [Language2]

Example 1 (Portuguese input - "Brasil"):

Question: Estou no Brasil, me fale sobre o Japão

Thought: Detectei PORTUGUÊS (palavra "Brasil" + "Estou"). Tenho {iter_max} iterações. Plano: (1) Brasil, (2) Japão, (3) moeda, (4) resposta com timezone manual.
Action: get_country_info: Brazil
PAUSE

Observation: Country: Brazil, Capital: Brasília, Population: 213M, Currency: BRL, Timezone: UTC-03:00, Area: 8.5M km²

Thought: Iteração 2 de {iter_max}. Buscando Japão.
Action: get_country_info: Japan
PAUSE

Observation: Country: Japan, Capital: Tokyo, Population: 125M, Currency: JPY, Timezone: UTC+09:00, Area: 377K km²

Thought: Iteração 3 de {iter_max}. Convertendo moeda.
Action: convert_currency: BRL, JPY
PAUSE

Observation: 1 BRL = 30.2156 JPY

Thought: Iteração 4 de {iter_max}. Tenho timezone dos países. Cálculo manual: UTC+09:00 - UTC-03:00 = 12 horas. Se 12h no Brasil, são 00h (meia-noite) no Japão. Finalizando.
Answer: Comparação Brasil vs Japão

Moeda: 1 Real = 30,22 Ienes
Horário: Quando são 12h em Brasília, são 00h em Tóquio (12 horas à frente)
Comparação:
- Japão é 22,5x menor em área (377K km² vs 8,5M km²)
- Brasil tem 1,7x mais habitantes (213M vs 125M)
- Capitais: Brasília vs Tóquio
- Idiomas: Português vs Japonês

Example 2 (English input - "Brazil"):

Question: I'm in Brazil, tell me about Japan

Thought: Detected ENGLISH ("I'm in" + "Brazil" not "Brasil"). I have {iter_max} iterations. Plan: (1) Brazil, (2) Japan, (3) currency, (4) answer with manual timezone.
Action: get_country_info: Brazil
PAUSE

Observation: Country: Brazil, Capital: Brasília, Population: 213M, Currency: BRL, Timezone: UTC-03:00, Area: 8.5M km²

Thought: Iteration 2 of {iter_max}. Getting Japan.
Action: get_country_info: Japan
PAUSE

Observation: Country: Japan, Capital: Tokyo, Population: 125M, Currency: JPY, Timezone: UTC+09:00, Area: 377K km²

Thought: Iteration 3 of {iter_max}. Converting currency.
Action: convert_currency: BRL, JPY
PAUSE

Observation: 1 BRL = 30.2156 JPY

Thought: Iteration 4 of {iter_max}. I have timezones. Manual calculation: UTC+09:00 - UTC-03:00 = 12 hours. If noon in Brazil, it's midnight in Japan. Finalizing.
Answer: Brazil vs Japan Comparison

Currency: 1 Real = 30.22 Yen
Time: When it's noon in Brasília, it's midnight in Tokyo (12 hours ahead)
Comparison:
- Japan is 22.5x smaller (377K km² vs 8.5M km²)
- Brazil has 1.7x more people (213M vs 125M)
- Capitals: Brasília vs Tokyo
- Languages: Portuguese vs Japanese

Example 3 (Portuguese, LOW iterations):

Question: Estou no Brasil, me fale sobre o Japão

Thought: PORTUGUÊS detectado. ALERTA: apenas {iter_max}=3 iterações! Plano: (1) Brasil, (2) Japão, (3) resposta com cálculos manuais.
Action: get_country_info: Brazil
PAUSE

Observation: Country: Brazil, Capital: Brasília, Population: 213M, Currency: BRL (R$), Timezone: UTC-03:00, Area: 8.5M km²

Thought: Iteração 2 de 3. Buscando Japão.
Action: get_country_info: Japan
PAUSE

Observation: Country: Japan, Capital: Tokyo, Population: 125M, Currency: JPY (¥), Timezone: UTC+09:00, Area: 377K km²

Thought: Iteração 3 de 3 - ÚLTIMA! Tenho dados dos países. Cálculo manual timezone: +9 - (-3) = +12h. Se 12h Brasil = 00h Japão. Moeda: sem API, informo indisponível. Finalizando AGORA.
Answer: Comparação Brasil vs Japão

Moeda: Conversão de moeda indisponível (limite de iterações)
Horário: Quando são 12h em Brasília, são 00h em Tóquio (12 horas à frente)
Comparação:
- Brasil é 22,5x maior em área (8,5M km² vs 377K km²)
- Brasil tem 1,7x mais habitantes (213M vs 125M)
- Capitais: Brasília vs Tóquio
- Idiomas: Português vs Japonês
- Moedas: Real (R$) vs Iene (¥)

Now it's your turn:
""".strip()

**5.2 Ferramentas utilizadas**

In [ ]:
def get_country_info(country_name):
    """
    Busca informações completas de um país via REST Countries API
    
    Args:
        country_name (str): Nome do país em inglês (ex: "Brazil", "Japan", "France")
    
    Returns:
        str: Informações formatadas do país ou mensagem de erro
    
    Exemplo:
        >>> get_country_info("Brazil")
        Country: Brazil
        Capital: Brasília
        Population: 213,421,037
        Language: Portuguese
        Currency: Brazilian real (BRL) R$
        Timezone: UTC-03:00
        Area: 8,515,767 km²
    """
    import requests
    
    url = f"https://restcountries.com/v3.1/name/{country_name.strip()}"
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()[0]
        
        # Extrair dados principais
        capital = data['capital'][0] if data.get('capital') else "N/A"
        population = f"{data['population']:,}"
        languages = ", ".join(data.get('languages', {}).values())
        
        # Processar moeda
        currencies = data.get('currencies', {})
        if currencies:
            curr_code = list(currencies.keys())[0]
            curr_info = currencies[curr_code]
            currency = f"{curr_info['name']} ({curr_code})"
            if 'symbol' in curr_info:
                currency += f" {curr_info['symbol']}"
        else:
            currency = "N/A"
        
        # Timezone principal (primeiro da lista, geralmente da capital)
        timezone = data.get('timezones', ['UTC'])[0]
        
        # Área
        area = f"{data.get('area', 0):,.0f}"
        
        # Retornar formatado
        return f"""Country: {data['name']['common']}
Capital: {capital}
Population: {population}
Language: {languages}
Currency: {currency}
Timezone: {timezone}
Area: {area} km²"""
        
    except Exception as e:
        return f"ERROR: Could not fetch data for {country_name}"

In [ ]:
def convert_currency(from_code, to_code):
    """
    Converte 1 unidade da moeda origem para moeda destino usando Frankfurter API
    
    Args:
        from_code (str): Código ISO 4217 da moeda origem (ex: "BRL", "USD", "EUR")
        to_code (str): Código ISO 4217 da moeda destino (ex: "JPY", "GBP", "CNY")
    
    Returns:
        str: Taxa de conversão formatada ou mensagem de erro
    
    Exemplo:
        >>> convert_currency("BRL", "USD")
        1 BRL = 0.1723 USD
        
        >>> convert_currency("EUR", "JPY")
        1 EUR = 163.2547 JPY
    """
    import requests
    
    from_code = from_code.strip().upper()
    to_code = to_code.strip().upper()
    
    url = f"https://api.frankfurter.dev/v1/latest?base={from_code}&symbols={to_code}"
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        rate = data['rates'][to_code]
        
        return f"1 {from_code} = {rate:.4f} {to_code}"
        
    except Exception as e:
        return f"ERROR: Could not convert {from_code} to {to_code}"

In [ ]:
def calculate_timezone_diff(tz_origin, tz_destination):
    """
    Calcula que horas são no destino quando são 12:00 na origem
    
    Args:
        tz_origin (str): Timezone do país origem (ex: "UTC-03:00", "UTC+05:30")
        tz_destination (str): Timezone do país destino (ex: "UTC+09:00")
    
    Returns:
        str: Horário correspondente no destino ou mensagem de erro
    
    Exemplo:
        >>> calculate_timezone_diff("UTC-03:00", "UTC+09:00")
        12:00 in UTC-03:00 = 00:00 in UTC+09:00 (next day)
        
        >>> calculate_timezone_diff("UTC+00:00", "UTC-05:00")
        12:00 in UTC+00:00 = 07:00 in UTC-05:00
    """
    import re
    
    try:
        def extract_offset(tz):
            """Extrai offset em horas de uma string timezone"""
            if tz == "UTC" or tz == "UTC+00:00":
                return 0
            
            match = re.search(r'UTC([+-])(\d{1,2}):?(\d{2})?', tz)
            if match:
                sign = 1 if match.group(1) == '+' else -1
                hours = int(match.group(2))
                minutes = int(match.group(3) or 0)
                return sign * (hours + minutes / 60)
            return 0
        
        offset_origin = extract_offset(tz_origin)
        offset_dest = extract_offset(tz_destination)
        diff = offset_dest - offset_origin
        
        # Calcular horário no destino (assumindo 12:00 na origem)
        hour_dest = (12 + diff) % 24
        
        # Determinar se é dia seguinte ou anterior
        if diff >= 12:
            day_note = " (next day)"
        elif diff <= -12:
            day_note = " (previous day)"
        else:
            day_note = ""
        
        return f"12:00 in {tz_origin} = {int(hour_dest):02d}:00 in {tz_destination}{day_note}"
        
    except Exception as e:
        return f"ERROR: Could not calculate timezone difference"

# 6. Implementando looping automático

In [ ]:
import re
from datetime import datetime


def agent_loop(max_iterations, system, current_country, destination_country, query="", 
               log_file="agent_log.txt"):
    """
    Loop principal do agente de comparação de países com log em arquivo
    
    Args:
        max_iterations (int): Limite máximo de iterações
        system (str): System prompt que define o comportamento do agente
        current_country (str): País atual do usuário
        destination_country (str): País de destino
        query (str): Query customizada (opcional)
        log_file (str): Nome do arquivo para salvar o log completo
    
    Returns:
        str: Resposta final do agente
    """
    
    # Criar o agente com a conexão e system prompt
    agent = Agent(client, system)
    
    # Lista de ferramentas disponíveis para o agente
    tools = ['get_country_info', 'convert_currency', 'calculate_timezone_diff']
    
    if query:
        next_prompt = query
    else:    
        # Formatar query inicial com os países informados
        next_prompt = f"I'm in {current_country}, tell me about {destination_country}"
    
    # Contador de iterações
    i = 0
    
    # Variável para armazenar resultado final
    final_result = ""
    
    # Abrir arquivo de log para escrita
    with open(log_file, 'w', encoding='utf-8') as f:
        # Cabeçalho do log
        f.write("=" * 80 + "\n")
        f.write("LOG COMPLETO DO AGENTE DE COMPARACAO DE PAISES\n")
        f.write("=" * 80 + "\n")
        f.write(f"Data/Hora: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Pais Atual: {current_country}\n")
        f.write(f"Pais Destino: {destination_country}\n")
        f.write(f"Limite de Iteracoes: {max_iterations}\n")
        f.write("=" * 80 + "\n\n")
        
        # Mostrar no console apenas informação básica
        print("=" * 80)
        print("AGENTE DE COMPARACAO DE PAISES")
        print("=" * 80)
        print(f"Pais Atual: {current_country}")
        print(f"Pais Destino: {destination_country}")
        print(f"Limite de Iteracoes: {max_iterations}")
        print(f"Processando...")
        
        # Loop principal de iterações
        while i < max_iterations:
            i += 1
            
            # Mostrar progresso no console
            print(f"  [{i}/{max_iterations}]", end=" ")
            
            # Log da iteração no arquivo
            f.write("\n" + "=" * 80 + "\n")
            f.write(f"ITERACAO: {i} de {max_iterations}\n")
            f.write("=" * 80 + "\n\n")
            
            # Obter resposta do modelo
            result = agent(next_prompt)
            
            # Salvar resultado completo no log
            f.write("RESPOSTA DO MODELO:\n")
            f.write("-" * 80 + "\n")
            f.write(result + "\n")
            f.write("-" * 80 + "\n\n")
            
            # Verificar se precisa executar alguma ferramenta
            if "PAUSE" in result and "Action" in result:
                # Extrair a ação e argumentos usando regex
                action = re.findall(r"Action:\s*([a-z_]+):\s*(.+)", result, re.IGNORECASE)
                
                if action:
                    chosen_tool = action[0][0].strip()
                    arg = action[0][1].strip()
                    
                    print(f"{chosen_tool}({arg})")
                    
                    # Log da ferramenta escolhida
                    f.write(f"FERRAMENTA ESCOLHIDA: {chosen_tool}\n")
                    f.write(f"ARGUMENTOS: {arg}\n\n")
                    
                    # Verificar se a ferramenta existe
                    if chosen_tool in tools:
                        try:
                            # Executar a ferramenta apropriada
                            if chosen_tool == 'get_country_info':
                                result_tool = get_country_info(arg)
                            
                            elif chosen_tool == 'convert_currency':
                                args = [a.strip() for a in arg.split(',')]
                                if len(args) == 2:
                                    result_tool = convert_currency(args[0], args[1])
                                else:
                                    result_tool = "ERROR: convert_currency requires 2 arguments"
                            
                            elif chosen_tool == 'calculate_timezone_diff':
                                args = [a.strip() for a in arg.split(',')]
                                if len(args) == 2:
                                    result_tool = calculate_timezone_diff(args[0], args[1])
                                else:
                                    result_tool = "ERROR: calculate_timezone_diff requires 2 arguments"
                            
                            # Preparar próximo prompt com o resultado da ferramenta
                            next_prompt = f"Observation: {result_tool}"
                            
                            # Salvar resultado da ferramenta no log
                            f.write("RESULTADO DA FERRAMENTA:\n")
                            f.write("-" * 80 + "\n")
                            f.write(result_tool + "\n")
                            f.write("-" * 80 + "\n\n")
                        
                        except Exception as e:
                            # Tratar erro na execução da ferramenta
                            error_msg = f"Error executing {chosen_tool}: {str(e)}"
                            next_prompt = f"Observation: {error_msg}"
                            
                            print(f"ERRO: {str(e)[:50]}")
                            
                            # Salvar erro no log
                            f.write("ERRO NA EXECUCAO DA FERRAMENTA:\n")
                            f.write("-" * 80 + "\n")
                            f.write(error_msg + "\n")
                            f.write("-" * 80 + "\n\n")
                    
                    else:
                        # Ferramenta não encontrada
                        next_prompt = "Observation: tool not found"
                        f.write("ERRO: Ferramenta nao encontrada\n\n")
                        print("Ferramenta nao encontrada")
                    
                    # Continuar para próxima iteração
                    continue
            
            # Verificar se chegou na resposta final
            if "Answer:" in result or "Answer" in result:
                print("CONCLUIDO")
                
                # Salvar indicação de conclusão no log
                f.write("\n" + "=" * 80 + "\n")
                f.write("RESPOSTA FINAL OBTIDA\n")
                f.write("=" * 80 + "\n\n")
                
                # Extrair apenas a parte da resposta final
                answer_match = re.search(r"Answer:?\s*(.+)", result, re.DOTALL | re.IGNORECASE)
                if answer_match:
                    final_result = answer_match.group(1).strip()
                else:
                    final_result = result
                
                f.write("RESPOSTA FINAL:\n")
                f.write("-" * 80 + "\n")
                f.write(final_result + "\n")
                f.write("-" * 80 + "\n")
                
                break
        
        # Verificar se atingiu o limite de iterações sem concluir
        if i >= max_iterations and not final_result:
            print("LIMITE ATINGIDO - forcando resposta")
            
            f.write("\n" + "=" * 80 + "\n")
            f.write("LIMITE DE ITERACOES ATINGIDO - FORCANDO RESPOSTA\n")
            f.write("=" * 80 + "\n")
            
            # Forçar uma última chamada para obter resposta
            force_prompt = f"You have reached iteration {max_iterations} of {max_iterations}. This is your LAST iteration. Output your Answer NOW with whatever information you have collected."
            final_result = agent(force_prompt)
            
            # Tentar extrair Answer
            answer_match = re.search(r"Answer:?\s*(.+)", final_result, re.DOTALL | re.IGNORECASE)
            if answer_match:
                final_result = answer_match.group(1).strip()
            
            f.write(final_result + "\n")
    
    # Mostrar no console apenas a resposta final
    print("\n" + "=" * 80)
    print("RESPOSTA FINAL")
    print("=" * 80)
    print(final_result)
    print("\n" + "=" * 80)
    print(f"Log completo salvo em: {log_file}")
    print(f"Iteracoes utilizadas: {i}/{max_iterations}")
    print("=" * 80)
    
    return final_result

In [ ]:
# ============================================================================
# EXEMPLO DE USO
# ============================================================================

current_country= input("Me diga o seu pais de origem:\t")
destination_country= input("Me diga um pais que gostaria de saber mais:\t")
query= input('Você gostaria de fazer uma pergunta especifica comparando o seu pais de origuem a outro? Se sim, pode perguntar aqui, \nSenão, aperte Enter!\t')

resultado_final = agent_loop(
    max_iterations=iter_max ,                             # max_iterations (1º argumento)
    system=system_prompt,                          # system (2º argumento)
    current_country=current_country,               # current_country (3º argumento)
    destination_country=destination_country,       # destination_country (4º argumento)
    log_file=f"log_{current_country}_{destination_country}.txt"  # log_file (nomeado)
)